In [ ]:
import os
import json
import pandas as pd

In [27]:
json_dir = '/home/morrissimons/Desktop/avancerad_ml/datasets'
merged_data = []

for filename in os.listdir(json_dir):
    if filename.endswith('.json') and filename != '6_dataset_full.json':
        file_path = os.path.join(json_dir, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            if isinstance(data, list):
                merged_data.extend(data)  # Flatten list
            else:
                merged_data.append(data)

output_file = os.path.join(json_dir, '6_dataset_full.json')
with open(output_file, 'w') as file:
    json.dump(merged_data, file, indent=4)

In [28]:
# Convert merged_data list to a pandas DataFrame
df_merged = pd.DataFrame(merged_data)

pd